# Quantile Regression 
>__Auteurs__: Yani Bouaffad, Ryma Lakehal, Loic Sauton

In [ ]:
import numpy as np
import pandas as pd
import cvxpy as cvx
import matplotlib.pyplot as plt
from download import download
import statsmodels.formula.api as ols
import time
import statsmodels.api as sm
from pathlib import Path
import pandas as pd
from sklearn.linear_model import RidgeCV, Ridge
import ipywidgets as ip
%matplotlib inline
import seaborn as sns
sns.set(context='talk', palette='colorblind', style='darkgrid')


In this part, we were inspired by Joly Julien , Mohamed Sahardid et Anas El Benna's homework to import the dataset "Mesure journalière Région Occitanie Polluants Principaux" and also for camparing what we will foundwith thiers results. 

In [ ]:
url = "http://josephsalmon.eu/enseignement/datasets/Mesure_journaliere_Region_Occitanie_Polluants_Principaux.csv"
path_target = "datasets/Mesure_journaliere_Region_Occitanie_Polluants_Principaux.csv"
download(url, path_target, replace=True)
occ_raw = pd.read_csv(path_target)
occ_raw['date'] = pd.to_datetime(occ_raw['date_debut']).dt.to_period('M')

occ = occ_raw.dropna()
occ.describe()

In [ ]:
occ["nom_com"].value_counts()

### liste de ville

In [ ]:
ville = 'MONTPELLIER', 'TOULOUSE', 'AGDE'

df = occ[occ['nom_com'].isin(ville)]
df = df[df['polluant'] == 'O3']

df.date.unique()

df = df[df.date == '2018-07']
df = df[['nom_com', 'valeur_originale']]

vecteurs les relevés d'ozone des villes choisies

In [ ]:
valeur_o3 = []
len_villes = []

for i in range(len(ville)):

    o3_i = df[df['nom_com'] == ville[i]].values[:, 1]

    # valeurs de O3 des villes choisies
    valeur_o3 = np.append(valeur_o3, o3_i)

    # Nombre d'observations de chaque ville
    len_villes = np.append(len_villes, o3_i.shape)

X = np.zeros((int(np.sum(len_villes)), len(ville)))

for i in range(len(ville)):
    X[int(np.sum(len_villes[0:i])):int(np.sum(len_villes[0:i + 1])
                                       ), i] = np.ones(
                                           int(np.sum(len_villes[i])))

# Quantile regression

Traditionally, the linear regression model for calculating the mean takes the form
$$y_i=\beta_0 + \beta_1 x_{i1} + \dots + \beta_p x_{ip} \ \ i=1, \dots , n$$
where p is equal to the number of features in the equation and n is the number of data points. The best linear regression line is found by minimizing the mean square error, which is found with the equation
$$ 
$$

blabla je finirai plus tard 

## implementing quantile regression using CVXPY package

CVXPY is a Python-embedded modeling language for convex optimization problems. It allows us to express our problem in a natural way that follows the math, rather than in the restrictive standard form required by solvers.

In [ ]:
alphas = np.linspace(0.0, 1, 11)

In [ ]:
Y = valeur_o3
X = X

fit = cvx.Variable(X.shape[1])
alpha = cvx.Parameter(value=0.5)

f = cvx.sum_entries(0.5 * cvx.abs((Y - (fit.T * X))) + (alpha - 0.5) * (Y - (fit.T * X)))

objective = cvx.Minimize(f)
problem = cvx.Problem(objective)